In [2]:
import pandas as pd
import pyarrow
from pyspark.sql import SparkSession
from google.colab import drive
import numpy as np

In [3]:
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Si vous voulez utiliser les même chemin que moi, créez un raccourcit dans votre "mon drive" de mon dossier partagé "nexialog"
https://drive.google.com/drive/folders/1d-uDDNs_H3Y977kY4F9A0elEoUKUxyz6?usp=drive_link

In [4]:
chemin_donnees_brutes = "/content/drive/MyDrive/nexialog/250203_tests_fixe_dns_sah_202412_202501.parquet"

In [24]:
chemin_donnees_preprocessees = "/content/drive/MyDrive/nexialog/250203_tests_fixe_dns_sah_202412_202501.parquet"

In [ ]:
donnee_brutes = pd.read_parquet(chemin_donnees_brutes, engine="pyarrow").sort_values(by="date_hour")

# rapport des données

In [ ]:
! pip install ydata-profiling

In [ ]:
from ydata_profiling import ProfileReport
profil = ProfileReport(donnee_brutes)  # explorative=True active toutes les analyses


In [ ]:
profil.to_file("rapport_données_brutes.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

# exploration

In [ ]:
donnee_brutes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9964847 entries, 0 to 9964846
Data columns (total 24 columns):
 #   Column               Dtype         
---  ------               -----         
 0   date_hour            datetime64[ns]
 1   code_departement     object        
 2   olt_model            object        
 3   olt_name             object        
 4   peag_nro             object        
 5   boucle               object        
 6   dsp                  object        
 7   pebib                object        
 8   pop_dns              object        
 9   nb_test_dns          int64         
 10  avg_dns_time         float64       
 11  std_dns_time         float64       
 12  nb_test_scoring      int64         
 13  avg_latence_scoring  float64       
 14  std_latence_scoring  float64       
 15  avg_score_scoring    float64       
 16  std_score_scoring    float64       
 17  nb_client_total      int64         
 18  moment_journee       object        
 19  dep_pop_dns          objec

In [ ]:
len(donnee_brutes)

9964847

In [ ]:
len(donnee_brutes.drop_duplicates(["date_hour","olt_name"]))

8492270

In [ ]:
len(donnee_brutes.drop_duplicates(["date_hour","peag_nro"]))

4447318

In [ ]:
#
len(donnee_brutes[donnee_brutes['nb_client_total'] < 10])


4167808

# premier traitements


In [8]:
donnee_brutes.columns

Index(['date_hour', 'code_departement', 'olt_model', 'olt_name', 'peag_nro',
       'boucle', 'dsp', 'pebib', 'pop_dns', 'nb_test_dns', 'avg_dns_time',
       'std_dns_time', 'nb_test_scoring', 'avg_latence_scoring',
       'std_latence_scoring', 'avg_score_scoring', 'std_score_scoring',
       'nb_client_total'],
      dtype='object')

In [6]:
# ____________________________________________________________calcul feature pour avoir le moment de la journée
def definir_moment_journee(heure):
    if 0 <= heure < 6:
        return "nuit"
    elif 6 <= heure < 12:
        return "matin"
    elif 12 <= heure < 18:
        return "après-midi"
    else:
        return "soir"

donnee_brutes["moment_journee"] = donnee_brutes["date_hour"].dt.hour.apply(definir_moment_journee)

# Afficher un aperçu
print(donnee_brutes[["date_hour", "moment_journee"]].head())


      date_hour moment_journee
0    2024-12-01           nuit
3812 2024-12-01           nuit
3811 2024-12-01           nuit
3810 2024-12-01           nuit
3809 2024-12-01           nuit


In [7]:
#_____________________________________________ création featuture si le test dns est au même département que le PM_______________________________

departement_region = {
    # Auvergne-Rhône-Alpes
    "01": "Auvergne-Rhône-Alpes", "03": "Auvergne-Rhône-Alpes", "07": "Auvergne-Rhône-Alpes",
    "15": "Auvergne-Rhône-Alpes", "26": "Auvergne-Rhône-Alpes", "38": "Auvergne-Rhône-Alpes",
    "42": "Auvergne-Rhône-Alpes", "43": "Auvergne-Rhône-Alpes", "63": "Auvergne-Rhône-Alpes",
    "69": "Auvergne-Rhône-Alpes", "73": "Auvergne-Rhône-Alpes", "74": "Auvergne-Rhône-Alpes",
    # Bourgogne-Franche-Comté
    "21": "Bourgogne-Franche-Comté", "25": "Bourgogne-Franche-Comté", "39": "Bourgogne-Franche-Comté",
    "58": "Bourgogne-Franche-Comté", "70": "Bourgogne-Franche-Comté", "71": "Bourgogne-Franche-Comté",
    "89": "Bourgogne-Franche-Comté", "90": "Bourgogne-Franche-Comté",
    # Bretagne
    "22": "Bretagne", "29": "Bretagne", "35": "Bretagne", "56": "Bretagne",
    # Centre-Val de Loire
    "18": "Centre-Val de Loire", "28": "Centre-Val de Loire", "36": "Centre-Val de Loire",
    "37": "Centre-Val de Loire", "41": "Centre-Val de Loire", "45": "Centre-Val de Loire",
    # Corse
    "2A": "Corse", "2B": "Corse",
    # Grand Est
    "08": "Grand Est", "10": "Grand Est", "51": "Grand Est", "52": "Grand Est",
    "54": "Grand Est", "55": "Grand Est", "57": "Grand Est", "67": "Grand Est",
    "68": "Grand Est", "88": "Grand Est",
    # Hauts-de-France
    "02": "Hauts-de-France", "59": "Hauts-de-France", "60": "Hauts-de-France",
    "62": "Hauts-de-France", "80": "Hauts-de-France",
    # Île-de-France
    "75": "Île-de-France", "77": "Île-de-France", "78": "Île-de-France",
    "91": "Île-de-France", "92": "Île-de-France", "93": "Île-de-France",
    "94": "Île-de-France", "95": "Île-de-France",
    # Normandie
    "14": "Normandie", "27": "Normandie", "50": "Normandie",
    "61": "Normandie", "76": "Normandie",
    # Nouvelle-Aquitaine
    "16": "Nouvelle-Aquitaine", "17": "Nouvelle-Aquitaine", "19": "Nouvelle-Aquitaine",
    "23": "Nouvelle-Aquitaine", "24": "Nouvelle-Aquitaine", "33": "Nouvelle-Aquitaine",
    "40": "Nouvelle-Aquitaine", "47": "Nouvelle-Aquitaine", "64": "Nouvelle-Aquitaine",
    "79": "Nouvelle-Aquitaine", "86": "Nouvelle-Aquitaine", "87": "Nouvelle-Aquitaine",
    # Occitanie
    "09": "Occitanie", "11": "Occitanie", "12": "Occitanie", "30": "Occitanie",
    "31": "Occitanie", "32": "Occitanie", "34": "Occitanie", "46": "Occitanie",
    "48": "Occitanie", "65": "Occitanie", "66": "Occitanie", "81": "Occitanie",
    "82": "Occitanie",
    # Pays de la Loire
    "44": "Pays de la Loire", "49": "Pays de la Loire", "53": "Pays de la Loire",
    "72": "Pays de la Loire", "85": "Pays de la Loire",
    # Provence-Alpes-Côte d'Azur
    "04": "Provence-Alpes-Côte d'Azur", "05": "Provence-Alpes-Côte d'Azur",
    "06": "Provence-Alpes-Côte d'Azur", "13": "Provence-Alpes-Côte d'Azur",
    "83": "Provence-Alpes-Côte d'Azur", "84": "Provence-Alpes-Côte d'Azur",
    # Départements d'outre-mer
    "971": "Guadeloupe", "972": "Martinique", "973": "Guyane", "974": "La Réunion", "976": "Mayotte"
}

donnee_brutes["dep_pop_dns"] = donnee_brutes["pop_dns"].str[:2]

# Vérifier si les départements sont égaux
donnee_brutes["depts_egaux"] = donnee_brutes["dep_pop_dns"] == donnee_brutes["code_departement"]

# Vérifier si les départements appartiennent à la même région
donnee_brutes["region_pop_dns"] = donnee_brutes["dep_pop_dns"].map(departement_region)
donnee_brutes["region_pm"] = donnee_brutes["code_departement"].map(departement_region)
donnee_brutes["meme_region"] = donnee_brutes["region_pop_dns"] == donnee_brutes["region_pm"]

# Afficher un aperçu
print(donnee_brutes[["code_departement", "pop_dns", "dep_pop_dns", "depts_egaux", "meme_region"]].head())


     code_departement  pop_dns dep_pop_dns  depts_egaux  meme_region
0                  01  69_lyon          69        False         True
3812               64   33_bdx          33        False         True
3811               64   33_bdx          33        False         True
3810               64   33_bdx          33        False         True
3809               64   33_bdx          33        False         True


In [14]:
#___________________________________ supression ligne si pas beaucoup de client au niveau PM (en moyenne il y a 25 clients par PM)

donnee_brutes_sans_pm_avec_peu_de_clients = donnee_brutes[donnee_brutes['nb_client_total'] > 15]
assert len(donnee_brutes_sans_pm_avec_peu_de_clients.drop_duplicates(["date_hour"])) ==  len(donnee_brutes.drop_duplicates(["date_hour"]))


In [15]:
# _________________________________________supression ligne si pas beaucoup de test au niveau des PM

seuil_nombre_de_test_minimum = 5

# Suppression des lignes où les deux colonnes sont en dessous du seuil
donnee_brutes_sans_pm_avec_peu_de_clients = donnee_brutes_sans_pm_avec_peu_de_clients[
    ~((donnee_brutes_sans_pm_avec_peu_de_clients["nb_test_scoring"] < seuil_nombre_de_test_minimum) &
      (donnee_brutes_sans_pm_avec_peu_de_clients["nb_test_dns"] < seuil_nombre_de_test_minimum))
]
donnee_brutes_sans_pm_avec_peu_de_clients.loc[
    donnee_brutes_sans_pm_avec_peu_de_clients["nb_test_dns"] < seuil_nombre_de_test_minimum,
    ["avg_dns_time", "std_dns_time"]
] = np.nan

# Remplacer par NaN si nb_test_scoring est sous le seuil
donnee_brutes_sans_pm_avec_peu_de_clients.loc[
    donnee_brutes_sans_pm_avec_peu_de_clients["nb_test_scoring"] < seuil_nombre_de_test_minimum,
    [ "avg_latence_scoring", "std_latence_scoring", "avg_score_scoring", "std_score_scoring"]
] = np.nan

donnee_brutes_sans_pm_avec_peu_de_clients_et_peu_de_tests = donnee_brutes_sans_pm_avec_peu_de_clients

# Afficher un aperçu
print(donnee_brutes_sans_pm_avec_peu_de_clients_et_peu_de_tests[
    ["nb_test_scoring", "nb_test_dns", "avg_dns_time", "std_dns_time",
     "avg_latence_scoring", "std_latence_scoring", "avg_score_scoring", "std_score_scoring"]
].head())

assert len(donnee_brutes_sans_pm_avec_peu_de_clients_et_peu_de_tests.drop_duplicates(["date_hour"])) ==  len(donnee_brutes.drop_duplicates(["date_hour"]))



      nb_test_scoring  nb_test_dns  avg_dns_time  std_dns_time  \
0                  25           61      4.888439      0.834768   
3811                4           17      3.583456      0.252478   
3810               10           10      3.638700      0.616376   
3802               12           20      4.808550      0.810622   
3800               13           27      4.637250      0.434874   

      avg_latence_scoring  std_latence_scoring  avg_score_scoring  \
0               10.462500             0.843750           2.852823   
3811                  NaN                  NaN                NaN   
3810             9.112500             0.237171           3.465882   
3802             9.796875             0.289647           3.404412   
3800             9.951923             0.270216           3.461539   

      std_score_scoring  
0              0.550380  
3811                NaN  
3810           0.005581  
3802           0.207982  
3800           0.004239  


In [22]:
donnee_brutes_sans_pm_avec_peu_de_clients.to_parquet(
    "/content/drive/MyDrive/nexialog/donnee_brutes_sans_pm_avec_peu_de_clients_et_peu_de_tests.parquet",
    engine="pyarrow",
    index=False
)


# agrégation et feature engenering

In [25]:
donnees_preprocessees = pd.read_parquet(chemin_donnees_preprocessees, engine="pyarrow").sort_values(by="date_hour")